In [1]:
import requests
import time
import numpy as np
import pandas as pd

from keras.layers import Input, Embedding, Flatten, Dot, Concatenate, Dropout, Dense
from keras.models import Model

Using TensorFlow backend.


# Third environmement

## Get The Data

In [5]:
#attributes fa
USER_ID = 'IAHIZPIW80WPGDW7P7JE'
BASE_URL = "http://35.180.178.243"
url_reset = BASE_URL + "/reset"
url_predict = BASE_URL + "/predict"
params = {"user_id" : USER_ID}

def load_data(url_reset,params) :
    r = requests.get(url=url_reset, params=params)
    data = r.json()
    return data

data = load_data(url_reset,params)
nb_users = data["nb_users"]
nb_items = data["nb_items"]
next_state = data["next_state"]

data.keys()

dict_keys(['action_history', 'nb_items', 'nb_users', 'next_state', 'rewards_history', 'state_history'])

In [6]:
# # #See the request predict from our API
# params = {"user_id" : user_id,"recommended_item": 2}
# requests.get(url=url_predict, params=params).json()

In [7]:
blacklisted_set = set(("next_state","nb_items","nb_users"))
new_dict = {key : value for key, value in data.items() if key not in blacklisted_set}

df = pd.DataFrame(new_dict)
df.head()

,action_history,rewards_history,state_history
0,10,0.000000,"[[5, 0, 995.4367425100899, -0.0168718993729635..."
1,14,0.000000,"[[58, 0, 995.4367425100899, 1.5616172497663623..."
2,4,647.932910,"[[92, 0, 995.4367425100899, 0.8019868269580972..."
3,6,0.000000,"[[41, 0, 995.4367425100899, 0.9170586867255156..."
4,23,178.207535,"[[94, 0, 995.4367425100899, 0.731733947245598,..."


In [8]:
df.describe()

,action_history,rewards_history
count,200.000000,200.000000
mean,14.895000,197.218886
std,8.629386,321.037754
min,0.000000,0.000000
25%,7.750000,0.000000
50%,15.000000,0.000000
75%,22.000000,336.974413
max,29.000000,995.436743


In [9]:
df[df['rewards_history']==0].count()

action_history     133
rewards_history    133
state_history      133
dtype: int64

In [10]:
users_items_df = []
for i in range(df.shape[0]):
    users_items_df += df['state_history'][i] 
    
users_items_df = pd.DataFrame(users_items_df,columns = ['user_id','item_id',
                                                        'price','v1','v2','v3','v4','v5'])
users_items_df

,user_id,item_id,price,v1,v2,v3,v4,v5
0,5,0,995.436743,-0.016872,0.248659,-0.122741,-0.859929,3.548400
1,5,1,268.197275,-0.016872,0.248659,1.804975,2.795424,2.349247
2,5,2,76.225432,-0.016872,0.248659,2.233305,0.514133,-0.756044
3,5,3,459.774176,-0.016872,0.248659,-0.976587,1.686933,0.222204
4,5,4,647.932910,-0.016872,0.248659,3.241630,2.866647,0.576906
5,5,5,119.735821,-0.016872,0.248659,-0.391206,0.835439,-1.345818
6,5,6,775.443156,-0.016872,0.248659,1.000608,0.572089,0.905308
7,5,7,35.822297,-0.016872,0.248659,0.160343,1.527066,0.430796
8,5,8,519.866211,-0.016872,0.248659,0.418672,-0.628954,-0.158545
9,5,9,743.797849,-0.016872,0.248659,-1.040818,1.852715,1.748791


In [11]:
# We can see that the first two covariates variables are features about users 
# and the forth and fifth variable are features about items
# The variable 5 depends on .....

user_v1 = users_items_df['v1'].unique()
user_v2 = users_items_df['v2'].unique()
users_ids = users_items_df['user_id'].unique()
users_content= pd.DataFrame({'user_id' : user_id,'v1':user_v1,'v2':user_v2 })

item_v3 = users_items_df['v3'].unique()
item_v4 = users_items_df['v4'].unique()
item_id = users_items_df['item_id'].unique()
items_content= pd.DataFrame({'item_id' : item_id,'v3':item_v3,'v4':item_v4 })

## Data Preprocessing

In [12]:
pos_data = df[df["rewards_history"] > 0].reset_index(drop=True)
neg_data = df[df["rewards_history"] == 0].reset_index(drop=True)

In [13]:
pos_data["user_id"] = [pos_data["state_history"][i][0][0] for i in range(pos_data.shape[0])]
pos_data.head()## There are some users that have more than one positive item

,action_history,rewards_history,state_history,user_id
0,4,647.932910,"[[92, 0, 995.4367425100899, 0.8019868269580972...",92
1,23,178.207535,"[[94, 0, 995.4367425100899, 0.731733947245598,...",94
2,19,193.390014,"[[89, 0, 995.4367425100899, 0.7959678436502442...",89
3,6,775.443156,"[[35, 0, 995.4367425100899, 1.0641933697598667...",35
4,4,647.932910,"[[94, 0, 995.4367425100899, 0.731733947245598,...",94


In [14]:
neg_data["user_id"] = [neg_data["state_history"][i][0][0] for i in range(neg_data.shape[0])]
neg_data.head()

,action_history,rewards_history,state_history,user_id
0,10,0.0,"[[5, 0, 995.4367425100899, -0.0168718993729635...",5
1,14,0.0,"[[58, 0, 995.4367425100899, 1.5616172497663623...",58
2,6,0.0,"[[41, 0, 995.4367425100899, 0.9170586867255156...",41
3,5,0.0,"[[76, 0, 995.4367425100899, 0.3964318617363663...",76
4,24,0.0,"[[76, 0, 995.4367425100899, 0.3964318617363663...",76


# Model

In [15]:
import tensorflow as tf

def identity_loss(y_true, y_pred):
    """Ignore y_true and return the mean of y_pred
    
    This is a hack to work-around the design of the Keras API that is
    not really suited to train networks with a triplet loss by default.
    """
    return tf.reduce_mean(y_pred + 0 * y_true)


def margin_comparator_loss(inputs, margin=1.):
    """Comparator loss for a pair of precomputed similarities
    
    If the inputs are cosine similarities, they each have range in
    (-1, 1), therefore their difference have range in (-2, 2). Using
    a margin of 1. can therefore make sense.

    If the input similarities are not normalized, it can be beneficial
    to use larger values for the margin of the comparator loss.
    """
    positive_pair_sim, negative_pair_sim = inputs
    return tf.maximum(negative_pair_sim - positive_pair_sim + margin, 0)

## Model 1

In [16]:
from keras.models import Model
from keras.layers import Embedding, Flatten, Input, Dense
from keras.layers import Lambda, Dot
from keras.regularizers import l2
from keras.layers.merge import dot, concatenate


def build_models(n_users, n_items, latent_dim=64, l2_reg=0):
    """Build a triplet model and its companion similarity model
    
    The triplet model is used to train the weights of the companion
    similarity model. The triplet model takes 1 user, 1 positive item
    (relative to the selected user) and one negative item and is
    trained with comparator loss.
    
    The similarity model takes one user and one item as input and return
    compatibility score (aka the match score).
    """
    # Common architectural components for the two models:
    # - symbolic input placeholders
    user_input = Input((1,), name='user_input')
    positive_item_input = Input((1,), name='positive_item_input')
    negative_item_input = Input((1,), name='negative_item_input')

    # - embeddings
    l2_reg = None if l2_reg == 0 else l2(l2_reg)
    user_layer = Embedding(input_dim=n_users, output_dim=latent_dim, input_length=1,
                           name='user_embedding', embeddings_regularizer=l2_reg)
    
    # The following embedding parameters will be shared to encode both
    # the positive and negative items.
    item_layer = Embedding(input_dim=n_items, output_dim=latent_dim, input_length=1,
                           name="item_embedding", embeddings_regularizer=l2_reg)

    user_embedding = Flatten()(user_layer(user_input))
    positive_item_embedding = Flatten()(item_layer(positive_item_input))
    negative_item_embedding = Flatten()(item_layer(negative_item_input))

    # - similarity computation between embeddings
    positive_similarity = Dot(name="positive_similarity",
                              axes=1, normalize=True)(
        [user_embedding, positive_item_embedding])
    negative_similarity = Dot(name="negative_similarity",
                              axes=1, normalize=True)(
        [user_embedding, negative_item_embedding])

    # The triplet network model, only used for training
    triplet_loss = Lambda(margin_comparator_loss,
                          name='comparator_loss',
                          output_shape=(1,))([positive_similarity, negative_similarity])

    triplet_model = Model(inputs=[user_input,
                                  positive_item_input,
                                  negative_item_input],
                          outputs=triplet_loss)
    
    # The match-score model, only use at inference to rank items for a given
    # model: the model weights are shared with the triplet_model therefore
    # we do not need to train it and therefore we do not need to plug a loss
    # and an optimizer.
    match_model = Model(inputs=[user_input, positive_item_input],
                        outputs=positive_similarity)
    
    return triplet_model, match_model


In [17]:
triplet_model, match_model = build_models(nb_users, nb_items, latent_dim = 16,l2_reg = 1e-6)

In [18]:
def sample_triplets(pos_data,max_items):
    """
    Sample negatives at random
    
    """
    range_itens = np.arange(max_items)
    user_ids = pos_data['user_id'].values
    pos_item_ids = pos_data['action_history'].values
    
    
    user_pos_items = pos_data.groupby('user_id')['action_history'].apply(list)
    neg_item_ids = np.array([])
    for i in user_ids:
        if i in set(user_pos_items.index):
            number = np.random.choice([i for i in range_itens 
                                       if not i in user_pos_items[user_pos_items.index == i]])
        else:
            number = np.random.choice(range_itens,1)
        neg_item_ids = np.append(neg_item_ids,number)
    
  
    return [user_ids, pos_item_ids, neg_item_ids]

In [19]:
# we plug the identity loss and the a fake target variable ignored by
# the model to be able to use the Keras API to train the triplet model
triplet_model.compile(loss=identity_loss, optimizer="adam")
fake_y = np.ones_like(pos_data['user_id'])
params = {'user_id' : USER_ID}
n_epochs = 15
total_reward = 0

generations = 10
positive_data = pos_data

for j in range(generations):
    
    for i in range(n_epochs):
        # Sample new negatives to build different triplets at each epoch
        triplet_inputs = sample_triplets(positive_data,max_items = nb_items)

        # Fit the model incrementally by doing a single pass over the
        # sampled triplets.
        triplet_model.fit(triplet_inputs, fake_y, shuffle=True,epochs=1)
    
    items_ids = np.array([next_state[i][1] for i in range(len(next_state))])
    repeated_user_id  = np.empty_like(items_ids)
    repeated_user_id.fill(next_state[0][0])
    
    predicted = match_model.predict([repeated_user_id,items_ids])
    
    predicted_item = np.argmax(predicted)
    params['recommended_item'] = predicted_item
    
    r = requests.get(url = url_predict, params=params).json()
    reward = r['reward']
    total_reward += reward
    
    if(reward > 0):
        positive_data = positive_data.append({'action_history':predicted_item,
                                              'state_history':next_state,
                                              'user_id':next_state[0][0]},
                                               ignore_index=True)
        
    fake_y = np.ones_like(positive_data['user_id'])
    next_state = r['state']

print("Total reward for each epoch :",total_reward/generations)

Epoch 1/1
67/67 [==============================] - 0s 6ms/step - loss: 1.0482
Epoch 1/1
67/67 [==============================] - 0s 88us/step - loss: 0.8868
Epoch 1/1
67/67 [==============================] - 0s 92us/step - loss: 0.8493
Epoch 1/1
67/67 [==============================] - 0s 105us/step - loss: 0.8529
Epoch 1/1
67/67 [==============================] - 0s 99us/step - loss: 0.7976
Epoch 1/1
67/67 [==============================] - 0s 114us/step - loss: 0.6929
Epoch 1/1
67/67 [==============================] - 0s 102us/step - loss: 0.7251
Epoch 1/1
67/67 [==============================] - 0s 106us/step - loss: 0.5833
Epoch 1/1
67/67 [==============================] - 0s 101us/step - loss: 0.6203
Epoch 1/1
67/67 [==============================] - 0s 85us/step - loss: 0.5403
Epoch 1/1
67/67 [==============================] - 0s 103us/step - loss: 0.5743
Epoch 1/1
67/67 [==============================] - 0s 107us/step - loss: 0.5172
Epoch 1/1
67/67 [=============================

Epoch 1/1
68/68 [==============================] - 0s 83us/step - loss: 0.0681
Epoch 1/1
68/68 [==============================] - 0s 105us/step - loss: 0.1387
15
Epoch 1/1
69/69 [==============================] - 0s 97us/step - loss: 0.0630
Epoch 1/1
69/69 [==============================] - 0s 134us/step - loss: 0.1155
Epoch 1/1
69/69 [==============================] - 0s 83us/step - loss: 0.1215
Epoch 1/1
69/69 [==============================] - 0s 88us/step - loss: 0.1311
Epoch 1/1
69/69 [==============================] - 0s 92us/step - loss: 0.0522
Epoch 1/1
69/69 [==============================] - 0s 89us/step - loss: 0.1062
Epoch 1/1
69/69 [==============================] - 0s 267us/step - loss: 0.0896
Epoch 1/1
69/69 [==============================] - 0s 130us/step - loss: 0.1498
Epoch 1/1
69/69 [==============================] - 0s 118us/step - loss: 0.1290
Epoch 1/1
69/69 [==============================] - 0s 104us/step - loss: 0.0697
Epoch 1/1
69/69 [==========================

## Model 2

In [52]:
from keras.models import Model, Sequential
from keras.layers import Embedding, Flatten, Input, Dense, Dropout
from keras.layers import Concatenate, Lambda
from keras.regularizers import l2


def make_interaction_mlp(input_dim, n_hidden=1, hidden_size=64,
                         dropout=0, l2_reg=None):
    """
    Build the shared multi layer perceptron
    
    """
    mlp = Sequential()
    if n_hidden == 0:
        # Plug the output unit directly: this is a simple
        # linear regression model. Not dropout required.
        mlp.add(Dense(1, input_dim=input_dim,
                      activation='relu', kernel_regularizer=l2_reg))
    else:
        mlp.add(Dense(hidden_size, input_dim=input_dim,
                      activation='relu', kernel_regularizer=l2_reg))
        mlp.add(Dropout(dropout))
        for i in range(n_hidden - 1):
            mlp.add(Dense(hidden_size, activation='relu',
                          kernel_regularizer=l2_reg))
            mlp.add(Dropout(dropout))
        mlp.add(Dense(1, activation='relu', kernel_regularizer=l2_reg))
    return mlp


def build_models(n_users, n_items, user_dim=32, item_dim=64,
                 n_hidden=1, hidden_size=64, dropout=0, l2_reg=0):
    """
    Build models to train a deep triplet network
    
    """
    user_input = Input((1,), name='user_input')
    positive_item_input = Input((1,), name='positive_item_input')
    negative_item_input = Input((1,), name='negative_item_input')
    positive_meta_input = Input((2,), name='positive_meta_input')
    negative_meta_input = Input((2,), name='negative_meta_input')

    l2_reg = None if l2_reg == 0 else l2(l2_reg)
    user_layer = Embedding(n_users, user_dim, input_length=1,
                           name='user_embedding', embeddings_regularizer=l2_reg)

    # The following embedding parameters will be shared to encode both
    # the positive and negative items.
    item_layer = Embedding(n_items, item_dim, input_length=1,
                           name="item_embedding", embeddings_regularizer=l2_reg)

    user_embedding = Flatten()(user_layer(user_input))
    positive_item_embedding = Flatten()(item_layer(positive_item_input))
    negative_item_embedding = Flatten()(item_layer(negative_item_input))


    # Similarity computation between embeddings using a MLP similarity
    positive_embeddings_pair = Concatenate(name="positive_embeddings_pair")(
        [user_embedding, positive_item_embedding,positive_meta_input])
    positive_embeddings_pair = Dropout(dropout)(positive_embeddings_pair)
    
    negative_embeddings_pair = Concatenate(name="negative_embeddings_pair")(
        [user_embedding, negative_item_embedding,negative_meta_input])
    negative_embeddings_pair = Dropout(dropout)(negative_embeddings_pair)

    # Instanciate the shared similarity architecture
    interaction_layers = make_interaction_mlp(
        user_dim + item_dim + 2, n_hidden=n_hidden, hidden_size=hidden_size,
        dropout=dropout, l2_reg=l2_reg)

    positive_similarity = interaction_layers(positive_embeddings_pair)
    negative_similarity = interaction_layers(negative_embeddings_pair)

    # The triplet network model, only used for training
    triplet_loss = Lambda(margin_comparator_loss, output_shape=(1,),
                          name='comparator_loss')(
        [positive_similarity, negative_similarity])

    deep_triplet_model = Model(inputs=[user_input,
                                       positive_item_input,
                                       negative_item_input,
                                       positive_meta_input,
                                       negative_meta_input
                                      ],
                               outputs=[triplet_loss])

    # The match-score model, only used at inference
    deep_match_model = Model(inputs=[user_input, positive_item_input,positive_meta_input],
                             outputs=[positive_similarity])

    return deep_match_model, deep_triplet_model

In [53]:
hyper_parameters = dict(
    user_dim=50,
    item_dim=16,
    n_hidden=1,
    hidden_size=16,
    dropout=0.2,
    l2_reg=1
)
deep_match_model, deep_triplet_model = build_models(nb_users, nb_items,
                                                    **hyper_parameters)

In [54]:
def sample_triplets(pos_data,items_content,max_items):
    """
    Sample the data and put in the format 
    [user_ids, pos_item_ids, neg_item_ids,pos_item_metadata,neg_item_metadata]
    
    """
    range_itens = np.arange(max_items)
    user_ids = pos_data['user_id'].values
    pos_item_ids = pos_data['action_history'].values
    
    pos_item_metadata = items_content.loc[pos_item_ids][['v3','v4']].values
    
    ##TODO
    user_pos_items = pos_data.groupby('user_id')['action_history'].apply(list)
    neg_item_ids = np.array([])
    for i in user_ids:
        if i in set(user_pos_items.index):
            number = np.random.choice([i for i in range_itens 
                                       if not i in user_pos_items[user_pos_items.index == i]])
        else:
            number = np.random.choice(range_itens,1)
        neg_item_ids = np.append(neg_item_ids,number)
    
    neg_item_metadata = items_content.loc[neg_item_ids][['v3','v4']].values
    
    
    return [user_ids, pos_item_ids, neg_item_ids,pos_item_metadata,neg_item_metadata]

In [55]:
# we plug the identity loss and the a fake target variable ignored by
# the model to be able to use the Keras API to train the triplet model
deep_triplet_model.compile(loss=identity_loss, optimizer="adam")
fake_y = np.ones_like(pos_data['user_id'])
params = {'user_id' : USER_ID}
n_epochs = 100
total_reward = 0
generations = 10

positive_data = pos_data

for j in range(generations):
    
    for i in range(n_epochs):

        # Sample new negatives to build different triplets at each epoch
        triplet_inputs = sample_triplets(positive_data,items_content,max_items = nb_items)

        # Fit the model incrementally by doing a single pass over the
        # sampled triplets.
        deep_triplet_model.fit(triplet_inputs, fake_y, shuffle=True,
                          batch_size=64, epochs=1)
    
    # Predicting 
    items_ids = np.array([next_state[i][1] for i in range(len(next_state))])
    repeated_user_id  = np.empty_like(items_ids)
    repeated_user_id.fill(next_state[0][0])
    
    pos_metadata = items_content.loc[items_ids][['v3','v4']].values

    predicted = deep_match_model.predict([repeated_user_id,items_ids,pos_metadata])
    
    predicted_item = np.argmax(predicted)
    params['recommended_item'] = predicted_item
    
    r = requests.get(url=url_predict, params=params).json()
    reward = r['reward']
    total_reward += reward
    
    if(reward > 0):
        positive_data = positive_data.append({'action_history':predicted_item,
                                              'state_history':next_state,
                                              'user_id':next_state[0][0]},
                                               ignore_index=True)
    
    fake_y = np.ones_like(positive_data['user_id'])
    next_state = r['state']

print("Total reward for each epoch :",total_reward/generations)

Epoch 1/1
67/67 [==============================] - 1s 16ms/step - loss: 33.0896
Epoch 1/1
67/67 [==============================] - 0s 76us/step - loss: 31.9975
Epoch 1/1
67/67 [==============================] - 0s 79us/step - loss: 30.9604
Epoch 1/1
67/67 [==============================] - 0s 82us/step - loss: 29.9301
Epoch 1/1
67/67 [==============================] - 0s 83us/step - loss: 28.9829
Epoch 1/1
67/67 [==============================] - 0s 75us/step - loss: 28.0754
Epoch 1/1
67/67 [==============================] - 0s 78us/step - loss: 27.2185
Epoch 1/1
67/67 [==============================] - 0s 82us/step - loss: 26.3803
Epoch 1/1
67/67 [==============================] - 0s 78us/step - loss: 25.6160
Epoch 1/1
67/67 [==============================] - 0s 131us/step - loss: 24.8658
Epoch 1/1
67/67 [==============================] - 0s 145us/step - loss: 24.1641
Epoch 1/1
67/67 [==============================] - 0s 76us/step - loss: 23.4919
Epoch 1/1
67/67 [=====================

67/67 [==============================] - 0s 101us/step - loss: 3.4498
Epoch 1/1
67/67 [==============================] - 0s 102us/step - loss: 3.3910
Epoch 1/1
67/67 [==============================] - 0s 77us/step - loss: 3.3336
Epoch 1/1
67/67 [==============================] - 0s 84us/step - loss: 3.2775
Epoch 1/1
67/67 [==============================] - 0s 90us/step - loss: 3.2229
Epoch 1/1
67/67 [==============================] - 0s 98us/step - loss: 3.1696
Epoch 1/1
67/67 [==============================] - 0s 93us/step - loss: 3.1177
Epoch 1/1
67/67 [==============================] - 0s 186us/step - loss: 3.0670
Epoch 1/1
67/67 [==============================] - 0s 87us/step - loss: 3.0176
Epoch 1/1
67/67 [==============================] - 0s 107us/step - loss: 2.9694
Epoch 1/1
67/67 [==============================] - 0s 76us/step - loss: 2.9225
Epoch 1/1
67/67 [==============================] - 0s 118us/step - loss: 2.8767
Epoch 1/1
67/67 [==============================] - 0s 93u

67/67 [==============================] - 0s 74us/step - loss: 1.2673
Epoch 1/1
67/67 [==============================] - 0s 87us/step - loss: 1.2627
Epoch 1/1
67/67 [==============================] - 0s 78us/step - loss: 1.2581
Epoch 1/1
67/67 [==============================] - 0s 137us/step - loss: 1.2537
Epoch 1/1
67/67 [==============================] - 0s 80us/step - loss: 1.2493
Epoch 1/1
67/67 [==============================] - 0s 90us/step - loss: 1.2451
Epoch 1/1
67/67 [==============================] - 0s 108us/step - loss: 1.2409
Epoch 1/1
67/67 [==============================] - 0s 176us/step - loss: 1.2368
Epoch 1/1
67/67 [==============================] - 0s 127us/step - loss: 1.2328
Epoch 1/1
67/67 [==============================] - 0s 85us/step - loss: 1.2289
Epoch 1/1
67/67 [==============================] - 0s 104us/step - loss: 1.2250
Epoch 1/1
67/67 [==============================] - 0s 84us/step - loss: 1.2213
Epoch 1/1
67/67 [==============================] - 0s 109

67/67 [==============================] - 0s 79us/step - loss: 1.0532
Epoch 1/1
67/67 [==============================] - 0s 84us/step - loss: 1.0523
Epoch 1/1
67/67 [==============================] - 0s 77us/step - loss: 1.0515
Epoch 1/1
67/67 [==============================] - 0s 88us/step - loss: 1.0508
Epoch 1/1
67/67 [==============================] - 0s 83us/step - loss: 1.0500
Epoch 1/1
67/67 [==============================] - 0s 76us/step - loss: 1.0492
Epoch 1/1
67/67 [==============================] - 0s 100us/step - loss: 1.0485
Epoch 1/1
67/67 [==============================] - 0s 85us/step - loss: 1.0477
Epoch 1/1
67/67 [==============================] - 0s 101us/step - loss: 1.0470
Epoch 1/1
67/67 [==============================] - 0s 79us/step - loss: 1.0463
Epoch 1/1
67/67 [==============================] - 0s 74us/step - loss: 1.0456
Epoch 1/1
67/67 [==============================] - 0s 90us/step - loss: 1.0449
Epoch 1/1
67/67 [==============================] - 0s 78us/s

67/67 [==============================] - 0s 77us/step - loss: 1.0099
Epoch 1/1
67/67 [==============================] - 0s 78us/step - loss: 1.0097
Epoch 1/1
67/67 [==============================] - 0s 74us/step - loss: 1.0095
Epoch 1/1
67/67 [==============================] - 0s 72us/step - loss: 1.0094
Epoch 1/1
67/67 [==============================] - 0s 77us/step - loss: 1.0092
Epoch 1/1
67/67 [==============================] - 0s 76us/step - loss: 1.0090
Epoch 1/1
67/67 [==============================] - 0s 81us/step - loss: 1.0089
Epoch 1/1
67/67 [==============================] - 0s 79us/step - loss: 1.0087
Epoch 1/1
67/67 [==============================] - 0s 81us/step - loss: 1.0086
Epoch 1/1
67/67 [==============================] - 0s 83us/step - loss: 1.0084
Epoch 1/1
67/67 [==============================] - 0s 87us/step - loss: 1.0083
Epoch 1/1
67/67 [==============================] - 0s 79us/step - loss: 1.0081
Epoch 1/1
67/67 [==============================] - 0s 76us/ste

67/67 [==============================] - 0s 85us/step - loss: 1.0014
Epoch 1/1
67/67 [==============================] - 0s 93us/step - loss: 1.0014
Epoch 1/1
67/67 [==============================] - 0s 74us/step - loss: 1.0013
Epoch 1/1
67/67 [==============================] - 0s 81us/step - loss: 1.0013
Epoch 1/1
67/67 [==============================] - 0s 81us/step - loss: 1.0013
Epoch 1/1
67/67 [==============================] - 0s 78us/step - loss: 1.0013
Epoch 1/1
67/67 [==============================] - 0s 192us/step - loss: 1.0012
Epoch 1/1
67/67 [==============================] - 0s 78us/step - loss: 1.0012
Epoch 1/1
67/67 [==============================] - 0s 87us/step - loss: 1.0012
Epoch 1/1
67/67 [==============================] - 0s 80us/step - loss: 1.0012
Epoch 1/1
67/67 [==============================] - 0s 78us/step - loss: 1.0011
Epoch 1/1
67/67 [==============================] - 0s 74us/step - loss: 1.0011
Epoch 1/1
67/67 [==============================] - 0s 79us/st

67/67 [==============================] - 0s 81us/step - loss: 1.0001
Epoch 1/1
67/67 [==============================] - 0s 198us/step - loss: 1.0001
Epoch 1/1
67/67 [==============================] - 0s 119us/step - loss: 1.0001
Epoch 1/1
67/67 [==============================] - 0s 82us/step - loss: 1.0001
Epoch 1/1
67/67 [==============================] - 0s 93us/step - loss: 1.0001
Epoch 1/1
67/67 [==============================] - 0s 94us/step - loss: 1.0001
Epoch 1/1
67/67 [==============================] - 0s 77us/step - loss: 1.0001
Epoch 1/1
67/67 [==============================] - 0s 85us/step - loss: 1.0001
Epoch 1/1
67/67 [==============================] - 0s 75us/step - loss: 1.0001
Epoch 1/1
67/67 [==============================] - 0s 187us/step - loss: 1.0001
Epoch 1/1
67/67 [==============================] - 0s 80us/step - loss: 1.0001
Epoch 1/1
67/67 [==============================] - 0s 113us/step - loss: 1.0001
Epoch 1/1
67/67 [==============================] - 0s 135u

67/67 [==============================] - 0s 128us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 90us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 155us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 117us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 131us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 75us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 123us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 81us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 137us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 86us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 91us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 77us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 98

67/67 [==============================] - 0s 91us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 88us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 71us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 84us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 85us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 81us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 81us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 79us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 81us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 100us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 75us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 80us/step - loss: 1.0000
Epoch 1/1
67/67 [==============================] - 0s 84us/st

68/68 [==============================] - 0s 74us/step - loss: 1.0000
Epoch 1/1
68/68 [==============================] - 0s 83us/step - loss: 1.0000
Epoch 1/1
68/68 [==============================] - 0s 113us/step - loss: 1.0000
Epoch 1/1
68/68 [==============================] - 0s 83us/step - loss: 1.0000
Epoch 1/1
68/68 [==============================] - 0s 85us/step - loss: 1.0000
Epoch 1/1
68/68 [==============================] - 0s 91us/step - loss: 1.0000
Epoch 1/1
68/68 [==============================] - 0s 81us/step - loss: 1.0000
Epoch 1/1
68/68 [==============================] - 0s 83us/step - loss: 1.0000
Epoch 1/1
68/68 [==============================] - 0s 74us/step - loss: 1.0000
Epoch 1/1
68/68 [==============================] - 0s 75us/step - loss: 1.0000
Epoch 1/1
68/68 [==============================] - 0s 75us/step - loss: 1.0000
Epoch 1/1
68/68 [==============================] - 0s 83us/step - loss: 1.0000
Epoch 1/1
68/68 [==============================] - 0s 83us/st